In [1]:
import pandas as pd
import numpy as np

In [8]:
#Step 1: Import document, Change Program Start Date to Date of Identification
file = 'BFZ_Report.csv'
df = pd.read_csv(file,parse_dates=['Program Start Date','Program End Date'])
df = df.rename(columns={"Program Start Date":"Date of Identification",'Case Number':'Client ID','Veteran Status (HUD)':'Veteran Status'})
df = df.drop(['Name','Family Name','Relationship','Gender (HUD)','Race (HUD)','Ethnicity (HUD)'],axis=1)
# df = df.drop(['Name'],axis=1)
df = df.drop_duplicates()
df = df.reset_index()
df = df.drop(['index'],axis=1)
df

,Age,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason
0,61.0,36939,Arlington Zero: Chronic or Veterans Only,No,2015-01-29,2016-02-25,Exit to Permanent Housing
1,58.0,38049,Arlington Zero: Chronic or Veterans Only,No,2021-02-15,2021-03-24,Exit to Permanent Housing
2,63.0,9330,Arlington Zero: Single Adults,No,2019-07-01,2020-06-30,NaN
3,44.0,7258,Arlington Zero: Chronic or Veterans Only,No,2015-02-02,2015-02-17,Service Completed Successfully
4,62.0,3464,Arlington Zero: Chronic or Veterans Only,No,2015-01-29,2015-08-24,Exit to Permanent Housing
...,...,...,...,...,...,...,...
634,68.0,55214,Arlington Zero: Single Adults,No,2023-05-16,NaT,NaN
635,33.0,55184,Arlington Zero: Single Adults,No,2023-05-08,NaT,NaN
636,43.0,4047,Arlington Zero: Chronic or Veterans Only,No,2018-07-01,2018-12-21,Exit to Permanent Housing
637,61.0,10128,Arlington Zero: Chronic or Veterans Only,No,2015-05-04,2015-05-05,NaN


In [3]:
#Step 2: Adding Household Type Column
household_type = {"Arlington Zero: Chronic or Veterans Only":"Single Adults",
"Arlington Zero: Single Adults":"Single Adults",
"Arlington Zero: Families":"Families",
"Arlington Zero: TAY":"Youth"}
df['Household Type'] = df['Program Name'].map(household_type)
df['Household Type'].value_counts()

Single Adults    537
Families          83
Youth             19
Name: Household Type, dtype: int64

In [5]:
#Step 3: Adding Client ID Counter and Client ID Household Counter
#Client ID Counter
df['Client ID Counter'] = df['Client ID'].map(df.groupby('Client ID').agg({'Client ID':'count'})['Client ID'])
#Client ID Household Counter
counter = {}
for i in df['Client ID']:
    if i not in counter:
        counter[i] = {"Single Adults":0,"Families":0,"Youth":0}
for j in counter:
    for k in df[df['Client ID']==j]['Household Type']:
        counter[j][k] += 1
df['Client ID Household Counter'] = np.nan
for i in df.index:
    df.loc[i,'Client ID Household Counter'] = counter[df.loc[i,'Client ID']][df.loc[i,'Household Type']]
df['Client ID Household Counter'] = df['Client ID Household Counter'].apply(int)
df['Client ID Household Counter'].value_counts()

1    527
2     88
3     24
Name: Client ID Household Counter, dtype: int64

In [6]:
#Step 4: Adding Chronic Column
#If client is in Chronic & Vet and 'No' to Veteran Status, then they are chronic
df['Chronic Status'] = np.nan
df.loc[(df['Program Name']=="Arlington Zero: Chronic or Veterans Only") & (df['Veteran Status']!="Yes")
,"Chronic Status"] = "Yes"